In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import rospy
import numpy as np
import sensor_msgs.point_cloud2 as pc2
from sensor_msgs.msg import PointCloud2
from std_msgs.msg import Header

class PointCloudTransformer:
    def __init__(self):
        rospy.init_node('pc_transformer_node', anonymous=True)

        # ---------------------------------------------------------
        # 1. 定义变换参数: T_cam_to_robot
        # ---------------------------------------------------------
        # 您提供的 4x4 矩阵拆解：
        # [ R00, R01, R02, Tx ]
        # [ R10, R11, R12, Ty ]
        # [ R20, R21, R22, Tz ]
        
        # 旋转矩阵 (3x3)
        self.R = np.array([
            [ 0.999864,  0.003046,  0.016227],
            [-0.006522, -0.830043,  0.557661],
            [ 0.015168, -0.557691, -0.82991 ]
        ])
        
        # 平移向量 (x, y, z)
        self.T = np.array([-0.452899, -0.662126, 0.395129])

        # ---------------------------------------------------------
        # 2. 初始化 ROS 订阅与发布
        # ---------------------------------------------------------
        self.sub = rospy.Subscriber('/camera/depth/points', PointCloud2, self.callback, queue_size=1)
        self.pub = rospy.Publisher('/camera/depth/points_transformed', PointCloud2, queue_size=1)
        
        rospy.loginfo("点云变换节点已启动 (Loaded custom T_cam_to_robot)...")

    def callback(self, msg):
        try:
            # --- 步骤 1: 解析原始点云 (N, 3) ---
            # skip_nans=True 会自动过滤掉深度无效(NaN)的点，防止计算报错
            points_gen = pc2.read_points(msg, field_names=("x", "y", "z"), skip_nans=True)
            points_np = np.array(list(points_gen))

            # 如果没有有效点，直接返回
            if points_np.shape[0] == 0:
                return

            # --- 步骤 2: 矩阵运算 (坐标变换) ---
            # 公式: P_new = R * P_old + T
            # 
            # 代码实现细节: 
            # points_np 是 (N, 3) 的矩阵，每行是一个点 [x, y, z]
            # 为了进行矩阵乘法，我们需要用 np.dot(points, R.T) 
            # (数学上等价于 (R * P_old^T)^T )
            
            transformed_points = np.dot(points_np, self.R.T) + self.T

            # --- 步骤 3: 发布转换后的点云 ---
            header = Header()
            header.stamp = msg.header.stamp
            
            # [重要]: 修改 frame_id
            # 因为我们应用了 T_cam_to_robot，现在的点应该是在 robot 坐标系下
            # 请将 'base_link' 修改为您实际机器人基座或世界坐标系的 frame_id 名称
            header.frame_id = "base_link" 
            
            # 创建新消息
            new_cloud = pc2.create_cloud_xyz32(header, transformed_points)
            self.pub.publish(new_cloud)

        except Exception as e:
            rospy.logerr(f"Error transforming point cloud: {e}")

if __name__ == '__main__':
    try:
        PointCloudTransformer()
        rospy.spin()
    except rospy.ROSInterruptException:
        pass

[INFO] [1766575276.215303]: 点云变换节点已启动 (Loaded custom T_cam_to_robot)...
